# 03 Yield Modeling (Y-band)Goal: build a **yield model** using climate + hazard features.We keep the last few years as the test set.

If features are missing, build them first:```bashpython src/data/merge_climate_daily.pypython src/data/compute_climate_hazards.pypython src/data/build_yield_highfreq.pypython src/features/build_dataset.py```

In [ ]:
import sysfrom pathlib import Pathimport matplotlib.pyplot as pltimport pandas as pdPROJECT_ROOT = Path("..").resolve()sys.path.append(str(PROJECT_ROOT))from src.models.yield_model import (    fit_baseline_mean,    fit_ml_models,    load_yield_features,    summarize_metrics,)

## 1) Load yield features

In [ ]:
crop = "wheat"data_dir = PROJECT_ROOT / "data"features = load_yield_features(crop=crop, data_dir=data_dir)features.head()

## 2) Quick look at the target

In [ ]:
if "year" in features.columns:    x = features["year"]else:    x = features.indextarget_col = "target_yield_t_ha" if "target_yield_t_ha" in features.columns else "yield_t_ha"plt.figure(figsize=(10, 4))plt.plot(x, features[target_col], marker="o")plt.title(f"Annual Yield - {crop.title()}")plt.xlabel("Year")plt.ylabel("Yield (t/ha)")plt.grid(True, alpha=0.3)plt.tight_layout()plt.show()

## 3) Train modelsBaseline mean vs ML models.

In [ ]:
baseline = fit_baseline_mean(crop=crop, test_size=4, data_dir=data_dir)ml_results = fit_ml_models(crop=crop, test_size=4, data_dir=data_dir)metrics = summarize_metrics([baseline, *ml_results.values()])metrics

## 4) Plot predictions (best ML model)

In [ ]:
metrics_ml = metrics[metrics["model"] != "baseline_mean"].reset_index(drop=True)best_model_name = metrics_ml.iloc[0]["model"]best_model = ml_results[best_model_name]plt.figure(figsize=(8, 4))plt.plot(best_model.test_index, best_model.y_true, label="actual", marker="o")plt.plot(best_model.test_index, best_model.y_pred, label=f"pred ({best_model_name})", marker="o")plt.title(f"Yield Predictions - {crop.title()}")plt.xlabel("Year")plt.ylabel("Yield (t/ha)")plt.grid(True, alpha=0.3)plt.legend()plt.tight_layout()plt.show()

## 5) Multi-crop comparison (optional)

In [ ]:
rows = []for crop_name in ["wheat", "barley", "maize"]:    baseline = fit_baseline_mean(crop=crop_name, test_size=4, data_dir=data_dir)    ml_results = fit_ml_models(crop=crop_name, test_size=4, data_dir=data_dir)    metrics = summarize_metrics([baseline, *ml_results.values()])    metrics.insert(0, "crop", crop_name)    rows.append(metrics)summary = pd.concat(rows, ignore_index=True)summary